In [21]:
import pickle
import numpy as np

from utils.ensemble import evaluate_dst_ensemble, dst_ensemble_predict

## Evaluation

In [22]:
val1 = np.load('./experiments/Binary1/val.npy')
val2 = np.load('./experiments/Binary2/val.npy')

feature1 = val1[:, :-1]
feature2 = val2[:, :-1]

label1 = val1[:, -1]
label2 = val2[:, -1]

if not np.array_equal(label1, label2):
    raise ValueError('Labels do not match')

model1 = pickle.load(open('./experiments/Binary1/model.pkl', 'rb'))
model2 = pickle.load(open('./experiments/Binary2/model.pkl', 'rb'))

/Users/huytrq/miniconda3/envs/py11/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [16:23:13] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


In [24]:
# Assuming ⁠ model1 ⁠ and ⁠ model2 ⁠ are trained XGBoost models with different feature sets
models = [model1, model2]
feature_sets = [feature1, feature2]
# # Evaluate the ensemble
ensemble_predictions = evaluate_dst_ensemble(models, feature_sets, label1)

DST Ensemble Accuracy: 0.8585353003161222
DST Ensemble Precision: 0.860392967942089
DST Ensemble Recall: 0.861729673744174
DST Ensemble F1 Score: 0.8610608020698577

Classification Report:
               precision    recall  f1-score   support

           A       0.86      0.86      0.86      1931
           B       0.86      0.86      0.86      1865

    accuracy                           0.86      3796
   macro avg       0.86      0.86      0.86      3796
weighted avg       0.86      0.86      0.86      3796



## Ensemble Prediction on Test Data

In [25]:
test1 = np.load('./experiments/Binary1/test.npy')
test2 = np.load('./experiments/Binary2/test.npy')

test_feature_sets = [test1, test2]

In [26]:
ensemble_predictions = dst_ensemble_predict(models, test_feature_sets)

# Map string labels to numeric for metric consistency, if necessary
label_map = {"A": 0, "B": 1}
numeric_predictions = [label_map[pred] for pred in ensemble_predictions]

## Export to csv

In [29]:
## Export results to CSV
import csv

csv_path = 'challenge1.csv'

with open(csv_path, mode='w') as file:
    writer = csv.writer(file)
    for result in numeric_predictions:
        writer.writerow([result])


In [30]:
import pandas as pd
df = pd.read_csv('./challenge1_single.csv', header=None)
df_new = pd.read_csv('./challenge1.csv', header=None)
old_results = df[0].values
new_results = df_new[0].values

# Count the number of indices with the same values in both lists
overlap_count = sum(1 for a, b in zip(old_results, new_results) if a == b)

print("Number of matching indices:", overlap_count/len(old_results))

Number of matching indices: 0.9582018927444795
